In [1]:
import numpy as np
import wordle

# Problem 1
def get_guess_result(true_word, guess):
    """
    Returns an array containing the result of a guess, with the return values as follows:
        2 - correct location of the letter
        1 - incorrect location but present in word
        0 - not present in word
    For example, if the true word is "boxed" and the provided guess is "excel", the 
    function should return [0,1,0,2,0].
    
    Arguments:
        true_word (string) - the secret word
        guess (string) - the guess being made
    Returns:
        result (array of integers) - the result of the guess, as described above
    """

    result = [0,0,0,0,0]     #create the results that we will return
    tru_letters = {i: true_word.count(i) for i in true_word}    #create dictionaries that indicate if we have addressed each letter's position
    guess_letters = {i: guess.count(i) for i in guess}

    guess = list(guess)
    true_word = list(true_word)         

    for i, let in enumerate(guess):
        if let == true_word[i]:          #if letter is in correct place assign result array a 2
            result[i] = 2
            true_word[i] = ''
            tru_letters[let] -= 1
            guess_letters[let] -= 1      #show that we have adressed the letter

    for i, let in enumerate(guess):
        if let in true_word and (result[i] != 2 and tru_letters[let] >= 1):
            result[i] = 1
            tru_letters[let] -= 1        #show that we have addressed the letter

    return result

In [2]:
def test1():
    # (0,1,0,2,0)
    true_word0 = "boxed"
    guess0 = "excel"
    print(true_word0, guess0, ": ", get_guess_result(true_word0, guess0))
    # (1,0,0,2,0)
    true_word1 = "pages"
    guess1 = "green"
    print(true_word1, guess1, ": ",get_guess_result(true_word1, guess1))
    # (0,1,0,0,0)
    true_word2 = "speak"
    guess2 = "bevel"
    print(true_word2, guess2, ": ", get_guess_result(true_word2, guess2))
    # (0,0,0,2,0)
    true_word3 = "ashes"
    guess3 = "bevel"
    print(true_word3, guess3, ": ",get_guess_result(true_word3, guess3))
    # (0,1,0,2,2)
    true_word4 = "steel"
    guess4 = "bevel"
    print(true_word4, guess4, ": ",get_guess_result(true_word4, guess4))
    # (0,1,0,1,0)
    true_word5 = "speae"
    guess5 = "bevel"
    print(true_word5, guess5, ": ",get_guess_result(true_word5, guess5))

In [3]:
test1()

boxed excel :  [0, 1, 0, 2, 0]
pages green :  [1, 0, 0, 2, 0]
speak bevel :  [0, 1, 0, 0, 0]
ashes bevel :  [0, 0, 0, 2, 0]
steel bevel :  [0, 1, 0, 2, 2]
speae bevel :  [0, 1, 0, 1, 0]


In [4]:
# Problem 2
def load_words(filen):
    """
    Loads all of the words from the given file, ensuring that they 
    are formatted correctly.
    """
    with open(filen, 'r') as file:
        # Get all 5-letter words
        words = [line.strip() for line in file.readlines() if len(line.strip()) == 5]
    return words

In [5]:
def get_all_guess_results(possible_words, allowed_words):
    """
    Calculates the result of making every guess for every possible secret word
    
    Arguments:
        possible_words (list of strings)
            A list of all possible secret words
        allowed_words (list of strings)
            A list of all allowed guesses
    Returns:
        ((n,m,5) ndarray) - the results of each guess for each secret word,
            where n is the the number
            of allowed guesses and m is number of possible secret words.
    """

    secret_combinations = np.array([np.array([get_guess_result(word, guess) for word in possible_words]) for guess in allowed_words])  #get each possible combination and use problem 2

    return secret_combinations

In [6]:
def save_stuff():
    possible_words = load_words('possible_words.txt')       #load the possible and allowed words

    allowed_words = load_words('allowed_words.txt')


    all_results = get_all_guess_results(possible_words, allowed_words)
    np.save('all_results.txt', all_results)     #save the results

In [7]:
# Problem 3
def compute_highest_entropy(all_guess_results, allowed_words):
    """
    Compute the entropy of each guess.
    
    Arguments:
        all_guess_results ((n,m,5) ndarray) - the output of the function
            from Problem 2, containing the results of each 
            guess for each secret word, where n is the the number
            of allowed guesses and m is number of possible secret words.
        allowed_words (list of strings) - list of the allowed guesses
    Returns:
        (string) The highest-entropy guess
        (int) Index of the highest-entropy guess
    """
    def ternary_gen(a, b, c, d, e):
        return a*1 + b*3 + c*9 + d*27 + e*81     #calculate the ternary generation of different arrays

    ter_list = ternary_gen(all_guess_results[:,:,0], all_guess_results[:, :, 1], all_guess_results[:, :, 2], all_guess_results[:,:,3], all_guess_results[:,:,4])   #calculate the ternary of each result

    entropies = []
    for ent in ter_list:
        dontcare, num_times = np.unique(ent, return_counts=True)     #find the number of occurrences each different value
        entropy = np.sum([num / len(ent) * -np.log2(num/len(ent)) for num in num_times])   #calculate the entropy of each entry
        entropies.append(entropy)

    max_ind = np.argmax(entropies)
    return allowed_words[max_ind], max_ind

In [8]:
def want_soare():
    hello = np.load('all_results.txt.npy')

    allowed = load_words('allowed_words.txt')

    print(compute_highest_entropy(hello, allowed))

In [9]:
want_soare()

('soare', 10349)


In [10]:
# Problem 4
def filter_words(all_guess_results, possible_words, guess_idx, result):
    """
    Create a function that filters the list of possible words after making a guess.
    Since we already computed the result of all guesses for all possible words in 
    Problem 2, we will use this array instead of recomputing the results.
    
	Return a filtered list of possible words that are still possible after 
    knowing the result of a guess. Also return a filtered version of the array
    of all guess results that only contains the results for the secret words 
    still possible after making the guess. This array will be used to compute 
    the entropies for making the next guess.
    
    Arguments:
        all_guess_results (3-D ndarray)
            The output of Problem 2, containing the result of making
            any allowed guess for any possible secret word
        possible_words (list of str)
            The list of possible secret words
        guess_idx (int)
            The index of the guess that was made in the list of allowed guesses.
        result (tuple of int)
            The result of the guess
    Returns:
        (list of str) The filtered list of possible secret words
        (3-D ndarray) The filtered array of guess results
    """
    filt_res = all_guess_results[:, np.all(all_guess_results[guess_idx] == result, axis=1), :]  #get the filtered array of guess results
    poss_secrets = np.array(possible_words)[np.all(all_guess_results[guess_idx] == result, axis=1)]      #use the guess results to get the list of possible secret words

    return poss_secrets, filt_res

In [11]:
def test4():
    get_all_guess_results = np.load('all_results.txt.npy')
    possible_words = load_words('possible_words.txt')

    guess_idx = 1914

    print(filter_words(get_all_guess_results, possible_words, guess_idx, [0,0,0,1,2]))

In [12]:
test4()

(array(['ivory', 'ovary', 'savoy'], dtype='<U5'), array([[[0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 2, 0, 0, 0]],

       [[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 2, 0, 0, 0]],

       [[0, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [0, 2, 0, 0, 0]],

       ...,

       [[0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 2, 0]],

       [[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1]],

       [[0, 1, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0]]]))


In [14]:
# Problem 5
def play_game_naive(game, all_guess_results, possible_words, allowed_words, word=None, display=False):
    """
    Plays a game of Wordle using the strategy of making guesses at random.
    
    Return how many guesses were used.
    
    Arguments:
        game (wordle.WordleGame)
            the Wordle game object
        all_guess_results ((n,m,5) ndarray)
            an array as outputted by problem 2 containing the results of every guess for every secret word.
        possible_words (list of str)
            list of possible secret words
        allowed_words (list of str)
            list of allowed guesses
        
        word (optional)
            If not None, this is the secret word; can be used for testing. 
        display (bool)
            If true, output will be printed to the terminal by the game.
    Returns:
        (int) Number of guesses made
    """
    # Initialize the game
    game.start_game(word=word, display=display)
    
    actual_word = game.word       #find the actual word that we are trying to find
    num_guesses = 0

    while len(possible_words) != 1:           #keep going until we only have one word aka the right one
        num_words = len(allowed_words)
        guess_idx = np.random.randint(0, num_words)    #choose a random guess
        rand_guess = allowed_words[guess_idx]
        res, num_guesses = game.make_guess(rand_guess)

        if actual_word == rand_guess:
            break
        possible_words, all_guess_results = filter_words(all_guess_results, possible_words, guess_idx, res)    #filter the results for next iteration
    return num_guesses

In [24]:
def test5():
    game = wordle.WordleGame()
    result = np.load('all_results.txt.npy')
    #output will be number of turns taken to get word
    print(play_game_naive(game, result, game.possible_words, game.allowed_words))

In [25]:
test5()

5


In [26]:
# Problem 6
def play_game_entropy(game, all_guess_results, possible_words, allowed_words, word=None, display=False):
    """
    Plays a game of Wordle using the strategy of guessing the maximum-entropy guess.
    
    Return how many guesses were used.
    
    Arguments:
        game (wordle.WordleGame)
            the Wordle game object
        all_guess_results ((n,m,5) ndarray)
            an array as outputted by problem 2 containing the results of every guess for every secret word.
        possible_words (list of str)
            list of possible secret words
        allowed_words (list of str)
            list of allowed guesses
        
        word (optional)
            If not None, this is the secret word; can be used for testing. 
        display (bool)
            If true, output will be printed to the terminal by the game.
    Returns:
        (int) Number of guesses made
    """
    # Initialize the game
    game.start_game(word=word, display=display)
    
    actual_word = game.word

    while len(possible_words) != 1:
        next_guess, guess_idx = compute_highest_entropy(all_guess_results, allowed_words)    #compute our next guess by entropy maximization
        res, num_guesses = game.make_guess(next_guess)

        if actual_word == next_guess:
            break

        possible_words, all_guess_results = filter_words(all_guess_results, possible_words, guess_idx, res)  #update our possible words by filtering

    return num_guesses

In [27]:
def test6():
    game = wordle.WordleGame()
    
    dat = np.load('all_results.txt.npy')
    #output will be number of turns taken to get word
    print(play_game_entropy(game, dat, game.possible_words, game.allowed_words))

In [28]:
test6()

2


In [29]:
# Problem 7
def compare_algorithms(all_guess_results, possible_words, allowed_words, n=20):
    """
    Compare the algorithms created in Problems 5 and 6. Play n games with each
    algorithm. Return the mean number of guesses the algorithms from
    problems 5 and 6 needed to guess the secret word, in that order.
    
    
    Arguments:
        all_guess_results ((n,m,5) ndarray)
            an array as outputted by problem 2 containing the results of every guess for every secret word.
        possible_words (list of str)
            list of possible secret words
        allowed_words (list of str)
            list of allowed guesses
        n (int)
            Number of games to run
    Returns:
        (float) - average number of guesses needed by naive algorithm
        (float) - average number of guesses needed by entropy algorithm
    """
    naive_guesses = []
    entropy_guesses = []

    for i in range(n):
        game = wordle.WordleGame()    #create a new game
        naive_guesses.append(play_game_naive(game, all_guess_results, possible_words, allowed_words, word=None, display=False))  #do naive way
        entropy_guesses.append(play_game_entropy(game, all_guess_results, possible_words, allowed_words, word=None, display=False))  #do entropy way
    return np.mean(naive_guesses), np.mean(entropy_guesses)

In [30]:
def test7():
    game = wordle.WordleGame()
    
    dat = np.load('all_results.txt.npy')
    #compare the average number of guesses needed by naive algorithm vs entropy algorithm
    print(compare_algorithms(dat, game.possible_words, game.allowed_words, 20))

In [31]:
#look at the difference!
test7()

(7.0, 2.65)
